In [174]:
import os
import glob
import urllib
import psycopg2
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
from numpy import NaN
from datetime import datetime  
from keras.models import load_model
from collections import OrderedDict
from sqlalchemy import create_engine

In [4]:
db_setting = pd.read_csv(os.getcwd()+'\\'+'set_file'+'\\'+'dbsetting.csv',encoding='ANSI') #database設定檔
time_setting = pd.read_csv(f'{os.getcwd()}'+'\\'+'set_file'+'\\'+'time.csv',encoding='ANSI') #讀取時間設定檔
hydro_setting = pd.read_csv(f'{os.getcwd()}'+'\\'+'set_file'+'\\'+'hydro_setting.csv',encoding='ANSI') #讀取氣象水文設定檔
farm = pd.read_csv(f'{os.getcwd()}'+'\\'+'tem'+'\\'+'farm_v3.csv',encoding='ANSI') #讀取坵塊資訊

In [7]:
#將坵塊資訊轉英文
farm.rename(columns={'處代碼':'placeID','工作站代碼':'workID','坵塊編號':'farmID','海拔高度(m)':'altitude','競用區':'conID','經度':'long','緯度':'lat','絕對最高氣溫(℃)':'tx01_max','絕對最低氣溫(℃)':'tx01_min','平均風速(m/s)':'wd01','相對溼度(%)':'rh01','平均日照時數':'ss01','累積降水量(mm)':'pp01','土壤質地':'soil','作物面積(平方公尺)':'area'},inplace=True)
farm['conID'] = farm['conID'].map(lambda x:'Liyutan' if x =='鯉魚潭水庫' else('Pao-Shan' if x=='寶山水庫' else ('Mingde' if x=='明德水庫' else('Ishioka' if x=='德基水庫(石岡壩)' else('Shimen' if x=='石門水庫' else ('wushantou' if x=='曾文-烏山頭' else x))))))
farm.drop('Unnamed: 0',axis=1,inplace=True)
#處理坵塊缺失/變數轉換
farm['soil'] = farm['soil'].replace('-',np.NaN)
farm[['placeID','workID','soil']] = farm[['placeID','workID','soil']].fillna(-9999)
farm[['placeID','workID','soil','altitude']] = farm[['placeID','workID','soil','altitude']].astype('int')

In [9]:
#connect db1
database1=db_setting['db1'][0]
user1 =db_setting['db1'][1]
passwords1=db_setting['db1'][2]
host1=db_setting['db1'][3]
port1=db_setting['db1'][4]
gssencmode1=db_setting['db1'][5]
engine1 = create_engine(f'postgresql+psycopg2://{user1}:%s@{host1}/{database1}?gssencmode={gssencmode1}'%urllib.parse.quote_plus(passwords1))
conn = psycopg2.connect(database=database1,
                        user=user1,
                        password=passwords1,
                        host=host1,
                        port=port1,
                        gssencmode=gssencmode1)
                        
#connect db2
database2=db_setting['db2'][0]
user2 =db_setting['db2'][1]
passwords2=db_setting['db2'][2]
host2=db_setting['db2'][3]
port2=db_setting['db2'][4]
gssencmode2=db_setting['db2'][5]
engine2 = create_engine(f'postgresql+psycopg2://{user2}:%s@{host2}/{database2}?gssencmode={gssencmode2}'%urllib.parse.quote_plus(passwords2))
conn = psycopg2.connect(database=database2,
                        user=user2,
                        password=passwords2,
                        host=host2,
                        port=port2,
                        gssencmode=gssencmode2)
                        

In [10]:
auto_column_list = ", ".join((hydro_setting['Input'][0],hydro_setting['Input'][2],hydro_setting['Input'][3]))
cwb_agr_column_list = ", ".join((hydro_setting['Input'][0],hydro_setting['Input'][1],hydro_setting['Input'][2],hydro_setting['Input'][3]))
qpesums_column_list = (hydro_setting['Input'][4])
qpesums_database_table = 'QPESUMS_history_grid_day'
cwb_database_table = 'cwb_history_hour'
agr_database_table = 'cwbagr_weather_obs_hr'
auto_database_table = 'cwb_history_auto_hour'
start_time = pd.to_datetime(str(time_setting['Input'][0]) + '_' + str(time_setting['Input'][1]) + '_' + str(time_setting['Input'][2]),format='%Y_%m_%d')
end_time = pd.to_datetime(str(time_setting['Input'][3]) + '_' + str(time_setting['Input'][4]) + '_' + str(time_setting['Input'][5]),format='%Y_%m_%d')
end_time = end_time.replace(hour=23, minute=0)
start_time_year = int(start_time.year)
end_time_year = int(end_time.year)

In [11]:
#build up the dic for Info(hour to day) to carry further calculation
day_input_methods ={}
for Input,Info in zip(hydro_setting['Input'],hydro_setting['Info(hour to day)']): 
    day_input_methods[Input] = Info

In [12]:
#build up the dic for Info(day to ten_days) to carry further calculation
ten_days_input_methods ={}
for Input,Info in zip(hydro_setting['Input'],hydro_setting['Info(day to ten_days)']): 
    ten_days_input_methods[Input] = Info

In [13]:
#hour to day calculation
def day_cal_variable(var_name: str, calculate_method: str) -> pd.DataFrame:
    if var_name in day_input_methods:
        if str(day_input_methods[var_name]) == '5':
            all_df[var_name] = df.groupby('stno').resample('D').sum()[var_name].values
        elif str(day_input_methods[var_name]) == '4':
            all_df[var_name] = df.groupby('stno').resample('D').mean()[var_name].values
        elif str(day_input_methods[var_name]) == '3':
            all_df[var_name+"_max"] = df.groupby('stno').resample('D').max()[var_name].values
        elif str(day_input_methods[var_name]) == '2':
            all_df[var_name+"_min"] = df.groupby('stno').resample('D').min()[var_name].values
        elif str(day_input_methods[var_name]) == '1':
            all_df[var_name+"_min"] = df.groupby('stno').resample('D').min()[var_name].values
            all_df[var_name+"_max"] = df.groupby('stno').resample('D').max()[var_name].values
            
#ten_days calculation
def ten_days_cal(df:pd.DataFrame(),var:str)->pd.DataFrame():
    if var =='tx01_min':
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)

    elif var =='tx01_max':
        df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
    else: 
        if ten_days_input_methods[var] == 5:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].sum(),2)
        elif ten_days_input_methods[var] == 4:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].mean(),2)
        elif ten_days_input_methods[var] == 3:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
        elif ten_days_input_methods[var] == 2:
            df[var] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)
        elif ten_days_input_methods[var] == 1:
            df[var+"_min"] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].min(),2)
            df[var+"_max"] = round(all_dff.groupby(['year','month','ten_days','stno'])[var].max(),2)
           

In [14]:
#build up the value dataset for each farm ID.

def data_filledIn(tem_df, index_num, stno, x: str) -> pd.DataFrame:
    if isinstance(x,str):
        if x[0] =='C':
            if index_num == 0:
                tem_df['tx01_max'] = auto_final_df[auto_final_df['stno'] == stno]['tx01_max'].reset_index(drop=True)
            elif index_num == 1:
                tem_df['tx01_min'] = auto_final_df.loc[auto_final_df['stno'] == stno, 'tx01_min'].reset_index(drop=True)
            elif index_num == 2:
                tem_df['wd01'] = auto_final_df.loc[auto_final_df['stno'] == stno, 'wd01'].reset_index(drop=True)
            else:
                tem_df['rh01'] = auto_final_df.loc[auto_final_df['stno'] == stno, 'rh01'].reset_index(drop=True)
        elif x[0] == '4':
            if index_num == 0:
                tem_df['tx01_max'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'tx01_max'].reset_index(drop=True)
            elif index_num == 1:
                tem_df['tx01_min'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'tx01_min'].reset_index(drop=True)
            elif index_num == 2:
                tem_df['wd01'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'wd01'].reset_index(drop=True)
            elif index_num == 3:
                tem_df['rh01'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'rh01'].reset_index(drop=True)
            else:
                tem_df['ss01'] = cwb_final_df.loc[cwb_final_df['stno'] == stno, 'ss01'].reset_index(drop=True)
        else:
            if index_num == 0:
                tem_df['tx01_max'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'tx01_max'].reset_index(drop=True)
            elif index_num == 1:
                tem_df['tx01_min'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'tx01_min'].reset_index(drop=True)
            elif index_num == 2:
                tem_df['wd01'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'wd01'].reset_index(drop=True)
            elif index_num == 3:
                tem_df['rh01'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'rh01'].reset_index(drop=True)
            else:
                tem_df['ss01'] = agr_final_df.loc[agr_final_df['stno'] == stno, 'ss01'].reset_index(drop=True)
    elif isinstance(x,np.int64):
        tem_df['pp01'] = qpesums_final_df.loc[qpesums_final_df['stno'] == stno, 'pp01'].reset_index(drop=True)
    
    tem_df['year'] = cwb_final_df[cwb_final_df['stno'] == '467050']['year'].reset_index(drop=True)
    tem_df['month'] = cwb_final_df[cwb_final_df['stno'] == '467050']['month'].reset_index(drop=True)
    tem_df['ten_days'] = cwb_final_df[cwb_final_df['stno'] == '467050']['ten_days'].reset_index(drop=True)
    tem_df.index = range(len(tem_df))

    return tem_df


In [15]:
# Get all the value we need from all the farmID. We stored them in each list for further retrieve purpose when build up data_filledIn this mentioned above.

auto_db =[]
cwb_db =[]
agr_db=[]
qpesums_db =[]

for i in set(farm['tx01_max']):
    if isinstance(i,str):
        if i[0] =='C':
            auto_db.append(i)    
        elif i[0] == '4':
            cwb_db.append(i)
        else :
            agr_db.append(i)
    else:
        qpesums_db.append(i)

for i in set(farm['ss01']):
    if isinstance(i,str):
        if i[0] =='C':
            auto_db.append(i)    
        elif i[0] == '4':
            cwb_db.append(i)
        else :
            agr_db.append(i)
    else:
        qpesums_db.append(i)

for i in set(farm['pp01']):
    if isinstance(i,str):
        if i[0] =='C':
            auto_db.append(i)    
        elif i[0] == '4':
            cwb_db.append(i)
        else :
            agr_db.append(i)
    else:
        qpesums_db.append(i)
auto_db = tuple(set(auto_db))
cwb_db =tuple(set(cwb_db))
agr_db=tuple(set(agr_db))
qpesums_db =tuple(set(qpesums_db))

#auto_db
query = f"select obsdate, obshour,stno, {auto_column_list} from {auto_database_table}_{start_time.year} where obsdate between '{start_time}' and '{end_time}' and (stno in {auto_db})"
df = pd.read_sql(query,con=engine1)
df['obstime'] = pd.to_datetime(df['obsdate'].astype(str)+'_'+df['obshour'].astype(str),format='%Y-%m-%d_%H')
df[df.isin([-9999,-9997,-9996,-9991])] = np.nan
df.set_index(df['obstime'],inplace=True)
all_df = pd.DataFrame()
auto_dict = {key: value for key, value in day_input_methods.items() if (key == hydro_setting['Input'][0]) | (key == hydro_setting['Input'][2]) | (key == hydro_setting['Input'][3])}
for j in auto_dict:
    day_cal_variable(f'{j}', f"{day_input_methods[j]}")
# add obstime and stno columns
all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
all_dff = all_df.copy()
all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_auto_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_auto_hydro_col = list(all_dff_auto_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_auto_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
auto_final_df = all_dff2.copy()


#cwb_db
query = f"select obsdate, obshour,stno, {cwb_agr_column_list} from {cwb_database_table}_{start_time.year} where obsdate between '{start_time}' and '{end_time}' and (stno in {cwb_db})"
df = pd.read_sql(query,con=engine1)
df['obstime'] = pd.to_datetime(df['obsdate'].astype(str)+'_'+df['obshour'].astype(str),format='%Y-%m-%d_%H')
df[df.isin([-9999,-9997,-9996,-9991,-9998])] = np.nan
df.set_index(df['obstime'],inplace=True)
all_df = pd.DataFrame()
cwb_dict = {key: value for key, value in day_input_methods.items() if (key == hydro_setting['Input'][0]) | (key == hydro_setting['Input'][1]) |(key == hydro_setting['Input'][2]) | (key == hydro_setting['Input'][3])}
for j in cwb_dict:
    day_cal_variable(f'{j}', f"{day_input_methods[j]}")
# add obstime and stno columns
all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
all_dff = all_df.copy()
all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_cwb_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_cwb_hydro_col = list(all_dff_cwb_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_cwb_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
cwb_final_df = all_dff2.copy()

#agr_db
query = f"select obstime,station, {cwb_agr_column_list} from {agr_database_table}_{start_time.year}_station where obstime between '{start_time}' and '{end_time}' and (station in {agr_db})"
df = pd.read_sql(query,con=engine2)
df.rename(columns={'station':'stno'},inplace=True)
df['obstime'] = pd.to_datetime(df['obstime'],format='%Y-%m-%d')
df['obshour'] = df['obstime'].dt.hour
df[df.isin([-9999,-9997,-9996,-9991,-9998])] = np.nan
df.set_index(df['obstime'],inplace=True)
all_df = pd.DataFrame()
agr_dict = {key: value for key, value in day_input_methods.items() if (key == hydro_setting['Input'][0]) | (key == hydro_setting['Input'][1]) |(key == hydro_setting['Input'][2]) | (key == hydro_setting['Input'][3])}
for j in agr_dict:
    day_cal_variable(f'{j}', f"{day_input_methods[j]}")
# add obstime and stno columns
all_df['obstime'] = df.groupby('stno').resample('D')['obstime'].first().values
all_df['stno'] = df.groupby('stno').resample('D')['stno'].first().values
all_dff = all_df.copy()
all_dff['obstime'] = pd.to_datetime(all_dff['obstime'], infer_datetime_format=True)
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_agr_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_agr_hydro_col = list(all_dff_agr_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_agr_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
agr_final_df = all_dff2.copy()

#qpesums_db
start_time_str = start_time.strftime("%Y-%m-%d")
end_time_str = end_time.strftime("%Y-%m-%d")
query = f'select grid,value,date from "{qpesums_database_table}_{start_time.year}_37134"' + f" where date between '{start_time_str}' and '{end_time_str}' and (grid in {qpesums_db})"
df = pd.read_sql(query,con=engine2)
df.rename(columns={'grid':'stno','value':'pp01'},inplace=True)
df['obstime'] = pd.to_datetime(df['date'].astype(str),format='%Y-%m-%d')
df.drop('date',axis=1,inplace=True)
df[df.isin([-9999,-9997,-9996,-9991,-9998])] = np.nan
all_dff = df.copy()
all_dff['year'] = all_dff['obstime'].dt.year
all_dff['month'] = all_dff['obstime'].dt.month
all_dff['day'] = all_dff['obstime'].dt.day
all_dff['ten_days'] = all_dff['day'].apply(lambda x:'1' if x <= 10 else ('2' if x <=20 else '3'))
all_dff.reset_index(drop=True,inplace=True)
all_dff_qpesums_hydro_col = all_dff.drop(['obstime', 'stno', 'year', 'month', 'day', 'ten_days'],axis=1)
all_dff_qpesums_hydro_col = list(all_dff_qpesums_hydro_col.columns)
all_dff2 = pd.DataFrame()
for i in all_dff_qpesums_hydro_col:
    ten_days_cal(all_dff2,i)
all_dff2.fillna(-9999,inplace=True)
all_dff2.reset_index(inplace=True)
qpesums_final_df = all_dff2.copy()

In [75]:
# Since farmID may vary each time we retrieve its ID, we will take its index to make sure we don't miss any farmID.
final_DF = pd.DataFrame() # Here we build a final dataframe to store all the value we got from each database (we did this above already ). 
tem_df_list = []
for i in tqdm(range(len(farm.index))):
    tem_df = pd.DataFrame() # This is a temporary dataframe which we use to store values from each index. 
    #Get the column values for tem_df 
    placeID = farm['placeID'].values[i]
    workID = farm['workID'].values[i]
    farmID = farm['farmID'].values[i]
    long = farm['long'].values[i]
    lat = farm['lat'].values[i]
    altitude = farm['altitude'].values[i]
    conID = farm['conID'].values[i]
    soil = farm['soil'].values[i]
    area = farm['area'].values[i]
    tx01_max = farm['tx01_max'].values[i]
    tx01_min = farm['tx01_min'].values[i]
    wd01 = farm['wd01'].values[i]
    rh01 = farm['rh01'].values[i]
    ss01 = farm['ss01'].values[i]
    pp01 = farm['pp01'].values[i]
    #Get the hydro values and stored them in a list 
    hydro_lst = [tx01_max, tx01_min, wd01, rh01, ss01, pp01]
    #we loop through every station values in hydro_lst to get the hydro values from the list which we already got from the database.
    for index, hydro_code in enumerate(hydro_lst):
        tem_df = data_filledIn(tem_df, index, hydro_code, hydro_code)
    tem_df['placeID'] = placeID
    tem_df['workID'] = workID
    tem_df['farmID'] = farmID
    tem_df['long'] = long
    tem_df['lat'] = lat
    tem_df['altitude'] = altitude
    tem_df['conID'] = conID
    tem_df['soil'] = soil
    tem_df['area'] = area
    tem_df_list.append(tem_df)
    
final_DF = pd.concat(tem_df_list)
final_DF.reset_index(drop=True)
#final_DF['conID'] = final_DF['conID'].map(lambda x:'鯉魚潭水庫' if x =='Liyutan' else('寶山水庫' if x=='Pao-Shan' else ('明德水庫' if x=='Mingde' else('德基水庫(石岡壩)' if x=='Ishioka' else('石門水庫' if x=='Shimen' else ('曾文-烏山頭' if x=='wushantou' else x))))))
final_DF.rename(columns={'year':'年','month':'月','ten_days':'旬','placeID':'處代碼','workID':'工作站代碼','farmID':'坵塊編號','altitude':'高程(m)','conID':'競用區','long':'經度','lat':'緯度','tx01_max':'絕對最高氣溫(℃)','tx01_min':'絕對最低氣溫(℃)','wd01':'平均風速(m/s)','rh01':'相對溼度(%)','ss01':'平均日照時數','pp01':'累積降水量(mm)','soil':'土壤質地','area':'作物面積(m2)'},inplace=True)
final_DF = final_DF[['年','月','旬','處代碼','工作站代碼','競用區','坵塊編號','經度','緯度','絕對最高氣溫(℃)','絕對最低氣溫(℃)','平均風速(m/s)','相對溼度(%)','平均日照時數','累積降水量(mm)','作物面積(m2)','高程(m)']]
file_year = final_DF['年'].iloc[0]
#final_DF.to_csv(f'{os.getcwd()}\\tem\\final_DF_{file_year}.csv')
Afinal_DF = final_DF.copy()
# if 判釋結果為1 作物面積(m2)==坵塊面積 else 作物面積(m2)==0

100%|██████████| 282911/282911 [33:30<00:00, 140.73it/s] 


In [ ]:
#check if the Afinal_DF contains any null values
print(Afinal_DF.isna().sum())

#if yes, show the null values
print(set(farm[farm['farmID'].isin(list(set(final_DF[final_DF['絕對最高氣溫(℃)'].isna()]['坵塊編號'])))]['tx01_max']))
print(set(farm[farm['farmID'].isin(list(set(final_DF[final_DF['絕對最高氣溫(℃)'].isna()]['坵塊編號'])))]['ss01']))

#the alternatives is to drop all the null
#Afinal_DF.dropna(how='any',axis=0).to_csv(f'{os.getcwd()}\\Console1\\INPUT\\\\INPUT.csv',encoding='ANSI',index=False)

In [399]:

path_dir = os.path.join(os.getcwd(), 'tem', '處理姜博資料')
file_patt = '*.txt'
file_path = glob.glob(os.path.join(path_dir, file_patt))

for i in file_path:
    with open(i, 'r', encoding='utf-8') as file:
        content = file.readlines()
        data = [line.strip().split(',') for line in content]
        column_names = data[0]
        data_without_column = data[1:]
        df = pd.DataFrame(data_without_column, columns=column_names)
        df.drop(columns=['管理處','工作站'], axis=1,inplace=True)
        df.to_csv(f"{os.getcwd()}\\Console1\\INPUT\\{i.split('.')[0][-3:]}.txt", index=False, header=True, sep=',')
    
#filter the data in Afinal_DF with the 姜博's data. We can't calculate the water usage for each farm without 姜博's data
sta_list=[]
for i in file_path:
    sta_list.append(int(i.split('.')[0][-3:]))

Afinal_DF = Afinal_DF[final_DF['工作站代碼'].isin(sta_list) == True]
Afinal_DF.to_csv(f'{os.getcwd()}\\Console1\\INPUT\\\\input.csv',encoding='ANSI')
Afinal_DF.dropna(how='any',axis=0).to_csv(f'{os.getcwd()}\\Console1\\INPUT\\\\input.csv',encoding='ANSI',index=False)
print('處代碼：',set(Afinal_DF['處代碼']))
print('工作站代碼：',set(Afinal_DF['工作站代碼']))
print('競用區：',set(Afinal_DF['競用區']))

處代碼： {3, 5, 6, 7}
工作站代碼： {708, 614, 615, 714, 301, 302, 303, 304, 305, 306, 307, 503}
競用區： {'明德水庫', '石門水庫', '德基水庫(石岡壩)', '鯉魚潭水庫', '寶山水庫'}


In [411]:
#exe progress
exe_path = f'{os.getcwd()}\\Console1\\Console1.exe'

# Run the executable file with arguments

os.system(f'cd {os.path.abspath(os.getcwd())}\\Console1 && Console1.exe')

0

In [456]:
STATION_OUT = pd.read_csv(f'{os.getcwd()}\\Console1\\output\\STATION_OUT.csv',encoding='utf-8', delim_whitespace=True)

In [ ]:
FARN = pd.read_csv(f'{os.getcwd()}\\Console1\\output\\BLOCK_OUT.csv',encoding='utf-8', delim_whitespace=True)


In [457]:
STATION_OUT

,年,月,旬,工作站,工作站蒸發散量(mm),工作站有效雨量(mm),工作站田間用水量(mm),工作站作物需水量(mm),工作站灌溉用水量(mm),工作站灌溉用水量(m3)
0,2022,1,12,708,2.142,4.903,0.00,0.0,0.0,0.0
1,2022,1,12,708,2.143,2.487,0.00,0.0,0.0,0.0
2,2022,1,12,708,1.590,30.056,0.00,0.0,0.0,0.0
3,2022,2,12,708,1.738,12.318,2.52,0.0,0.0,0.0
4,2022,2,12,708,1.720,37.183,6.12,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
427,2022,11,12,503,NaN,NaN,NaN,NaN,NaN,0.0
428,2022,11,12,503,NaN,NaN,NaN,NaN,NaN,0.0
429,2022,12,12,503,NaN,NaN,NaN,NaN,NaN,0.0
430,2022,12,12,503,NaN,NaN,NaN,NaN,NaN,0.0


In [426]:
OFFICE_OUT = pd.read_csv(f'{os.getcwd()}\\Console1\\output\\OFFICE_OUT.csv',encoding='utf-8', delim_whitespace=True)

In [429]:
OFFICE_OUT

,年,月,旬,管理處,管理處蒸發散量(mm),管理處有效雨量(mm),管理處田間用水量(mm),管理處作物需水量(mm),管理處灌溉用水量(mm),管理處灌溉用水量(m3)
0,2022,1,12,3,NaN,11.462,NaN,NaN,NaN,NaN
1,2022,1,12,3,NaN,7.296,NaN,NaN,NaN,NaN
2,2022,1,12,3,NaN,51.935,NaN,NaN,NaN,NaN
3,2022,2,12,3,NaN,58.789,NaN,NaN,NaN,NaN
4,2022,2,12,3,NaN,75.270,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
139,2022,11,12,7,2.838,0.070,0.204,1.256,1.875,58248.188
140,2022,11,12,7,2.577,6.401,0.000,0.000,0.000,0.000
141,2022,12,12,7,1.814,2.563,0.000,0.000,0.000,0.000
142,2022,12,12,7,1.811,6.060,0.000,0.000,0.000,0.000


In [ ]:
ZONE_OUT = pd.read_csv(f'{os.getcwd()}\\Console1\\output\\ZONE_OUT.csv', delim_whitespace=True, engine='python-fwf')

In [454]:
ZONE_OUT

,å¹´,æ,æ¬,ç«¶ç¨å,ç«¶ç¨åè¸ç¼æ£é(mm),ç«¶ç¨åææé¨é(mm),ç«¶ç¨åç°éç¨æ°´é(mm),ç«¶ç¨åä½ç©éæ°´é(mm),ç«¶ç¨åçæºç¨æ°´é(mm),ç«¶ç¨åçæºç¨æ°´é(m3)
0,2022,1,12,©ú¼w¤ô®w,1.993,5.571,0.0,0.0,0.0,0.0
1,2022,1,12,©ú¼w¤ô®w,1.973,4.475,0.0,0.0,0.0,0.0
2,2022,1,12,©ú¼w¤ô®w,1.526,46.883,0.0,0.0,0.0,0.0
3,2022,2,12,©ú¼w¤ô®w,1.572,29.937,0.0,0.0,0.0,0.0
4,2022,2,12,©ú¼w¤ô®w,1.616,46.963,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
175,2022,11,12,Ä_¤s¤ô®w,NaN,NaN,NaN,NaN,NaN,0.0
176,2022,11,12,Ä_¤s¤ô®w,NaN,NaN,NaN,NaN,NaN,0.0
177,2022,12,12,Ä_¤s¤ô®w,NaN,NaN,NaN,NaN,NaN,0.0
178,2022,12,12,Ä_¤s¤ô®w,NaN,NaN,NaN,NaN,NaN,0.0


In [449]:
before = ZONE_OUT['ç«¶ç¨å'][0]

In [450]:
after = before.encode("utf-8", errors="replace")

In [453]:
print(after.decode("utf-8"))

©ú¼w¤ô®w
